In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import sys

In [2]:
# Nombre del archivo PDB del péptido
pdb_filename = "pdbs/V5.pdb"

In [3]:
# Cargar la estructura del péptido desde el archivo PDB
pdb = PDBFile(pdb_filename)

In [4]:
# Definir la fuerza de campo (AMBER ff14SB es común para péptidos)
forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [5]:
# Crear el sistema en vacío (sin agua ni iones)
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=NoCutoff,  # Sin tratamiento de fronteras periódicas
    constraints=HBonds         # Restringe enlaces de hidrógeno para un paso de integración más largo
)

In [6]:
# Configurar el integrador (Langevin dynamics)
temperature = 300 * kelvin
friction = 1 / picosecond
timestep = 2 * femtoseconds
integrator = LangevinIntegrator(temperature, friction, timestep)

In [7]:
# Crear una plataforma para ejecutar la simulación (e.g., CUDA si tienes GPU)
platform = Platform.getPlatformByName("CUDA")  # Usa "CPU" si no tienes GPU

In [8]:
# Configurar la simulación
simulation = Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [9]:
# Minimización de energía
print("Minimizando energía...")
simulation.minimizeEnergy()

Minimizando energía...


In [10]:
# Configurar archivo de salida para las trayectorias
simulation.reporters.append(PDBReporter("output.pdb", 100))  # Guardar un frame cada 100 pasos
simulation.reporters.append(StateDataReporter(
    sys.stdout, 100, step=True, potentialEnergy=True, temperature=True
))

In [11]:
# Correr la simulación
print("Corriendo simulación...")
simulation.step(50000)  # 1,000 pasos de integración

Corriendo simulación...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,188.72710990905762,57.38095889247783
200,247.85805797576904,117.55165289874373
300,281.1056305170059,153.82855876969143
400,313.51591724157333,170.03424027390164
500,351.42987394332886,197.27604685167321
600,387.3434643149376,192.15424023569216
700,364.90098959207535,246.8860864478591
800,370.2626633644104,254.5078511954048
900,387.71325623989105,259.9372094028122
1000,353.5897091627121,310.0504312162163
1100,384.7094495296478,243.82740774808153
1200,374.42767310142517,258.82433970828936
1300,353.3694975376129,312.3084851372612
1400,393.7011845111847,312.69687346796553
1500,383.97796070575714,318.80501538669193
1600,355.4329252243042,297.63073782367985
1700,353.1008677482605,329.25432917135447
1800,379.623415350914,289.28339857022206
1900,355.29322469234467,320.85334010271725
2000,357.60920464992523,316.7584732901975
2100,318.17443096637726,313.8072004698467
2200,373.58826875686646,310.94260834964166
230

In [12]:
import nglview as nv
import mdtraj as md

In [13]:
traj = md.load('output.pdb')

In [14]:
nv.show_mdtraj(traj)

NGLWidget(max_frame=499)